<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Задание 3

## Классификация текстов

В этом задании вам предстоит попробовать несколько методов, используемых в задаче классификации, а также понять насколько хорошо модель понимает смысл слов и какие слова в примере влияют на результат.

In [ ]:
!pip install -q portalocker>=2.0.0
!pip install -q pytorch-lightning
!pip install -q captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import os
import portalocker
from math import ceil
from sklearn.metrics import f1_score
from torch.nn import Sigmoid

import torch

from torchtext import datasets
from torchtext.vocab import vocab

from collections import Counter

# from torchtext.legacy.data import Field, LabelField
# from torchtext.legacy.data import BucketIterator
from torchtext.data.utils import get_tokenizer
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import pytorch_lightning as ptl

from sklearn.model_selection import train_test_split

from torchtext.vocab import Vectors, GloVe

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from tqdm.autonotebook import tqdm, trange

В этом задании мы будем использовать библиотеку torchtext. Она довольна проста в использовании и поможет нам сконцентрироваться на задаче, а не на написании Dataloader-а.

In [ ]:
tokenizer = get_tokenizer('basic_english')

In [ ]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.random.manual_seed(SEED)
torch.cuda.random.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = False

Датасет на котором мы будем проводить эксперементы это комментарии к фильмам из сайта IMDB.

In [ ]:
train, test = datasets.IMDB(split=('train', 'test'))  # загрузим датасет
#train, valid = train.split(random_state=random.seed(SEED))  # разобьем на части

In [ ]:
train_list = []
for label, line in train:
  train_list.append((label, line))

In [ ]:
test_list = []
for label, line in test:
  test_list.append((label, line))

In [ ]:
train_list[0]

(1,
 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far betwee

In [ ]:
#train_list = list(train)
#test_list = list(test)

In [ ]:
len(test_list)

25000

In [ ]:
len(train_list)

25000

In [ ]:
#TEXT.build_vocab(train)
#LABEL.build_vocab(train)

In [ ]:
labels_to_text = {0: 'neg', 1: 'pos'}

In [ ]:
train_list[0]

(1,
 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far betwee

In [ ]:
train_labels = np.array([train_sample[0] for train_sample in train_list])
train_data = np.array([train_sample[1] for train_sample in train_list])

In [ ]:
train_labels

array([1, 1, 1, ..., 2, 2, 2])

In [ ]:
train_data

array(['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far betw

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    train_data, train_labels, stratify = train_labels, test_size = 0.2
)

In [ ]:
# Проверим, сбалансированы ли классы
for label in [1, 2]:
  print(sum([sample == label for sample in train_labels]) / len(train_labels))

0.5
0.5


In [ ]:
for label in [1, 2]:
   print(sum([sample == label for sample in y_valid]) / len(y_valid))

0.5
0.5


In [ ]:
len(y_valid)

5000

In [ ]:
train_list = [(y_train[i], X_train[i]) for i in range(len(y_train))]
valid_list = [(y_valid[i], X_valid[i]) for i in range(len(y_valid))]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
torch.version.cuda

'11.8'

In [ ]:
!python -m torch.utils.collect_env

In [ ]:
counter = Counter()
for (label, line) in train_list:
  counter.update(tokenizer(line))
vocabular = vocab(counter, min_freq=5, specials=('', "<unk>", "<pad>"))

In [ ]:
vocabular['<unk>']

1

In [ ]:
len(vocabular)

27014

In [ ]:
new_stoi = vocabular.get_stoi()
new_itos = vocabular.get_itos()

In [ ]:
new_stoi['<unk>']

1

In [ ]:
vocabular.set_default_index(new_stoi['<unk>'])

In [ ]:
text_transform = lambda x: [vocabular['']] + [vocabular[token] for token in tokenizer(x)] + [vocabular['']]
label_transform = lambda x: 1 if x == 2 else 0

print('output of text transform of "here is an example":', text_transform('here is an example'))

output of text transform of "here is an example": [0, 476, 26, 162, 123, 0]


In [ ]:
def collate_batch(batch):
  label_list, data_list = [], []
  for (label, text) in batch:
    label = label_transform(label)
    label_list.append(label)
    data = torch.tensor(text_transform(text))
    data_list.append(data)
  return torch.tensor(label_list), pad_sequence(data_list, padding_value=3.0)

In [ ]:
batch_size = 8
def train_batch_sampler():
    indices = [(i, len(tokenizer(s[1]))) for i, s in enumerate(train_list)]
    random.shuffle(indices)
    pooled_indices = []
    # create pool of indices with similar lengths
    for i in range(0, len(indices), batch_size * 100):
        pooled_indices.extend(sorted(indices[i:i + batch_size * 100],
                                     key=lambda x: x[1]))

    pooled_indices = [x[0] for x in pooled_indices]

    # yield indices for current batch
    for i in range(0, len(pooled_indices), batch_size):
        yield pooled_indices[i:i + batch_size]

def val_batch_sampler():
    indices = [(i, len(tokenizer(s[1]))) for i, s in enumerate(valid_list)]
    random.shuffle(indices)
    pooled_indices = []
    # create pool of indices with similar lengths
    for i in range(0, len(indices), batch_size * 100):
        pooled_indices.extend(sorted(indices[i:i + batch_size * 100],
                                     key=lambda x: x[1]))

    pooled_indices = [x[0] for x in pooled_indices]

    # yield indices for current batch
    for i in range(0, len(pooled_indices), batch_size):
        yield pooled_indices[i:i + batch_size]

In [ ]:
dataloaders = {
               k: DataLoader(
                   data_list,
                   batch_size=batch_size,
                   # batch_sampler=batch_sampler,
                   collate_fn=collate_batch) for k, data_list in zip(['train', 'val'],
                                                                     [train_list, valid_list])
                                                                     #[train_batch_sampler(), val_batch_sampler()])
}

In [ ]:
test_loader = DataLoader(test_list,
                         collate_fn=collate_batch,
                         batch_size=batch_size)

In [ ]:
'''
device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, valid_iter, test_iter = BucketIterator.splits(
    (train_list, valid_list, test_list),
    batch_size = 64,
    sort_within_batch = True,
    device = device)
'''

'\ndevice = "cuda" if torch.cuda.is_available() else "cpu"\n\ntrain_iter, valid_iter, test_iter = BucketIterator.splits(\n    (train_list, valid_list, test_list), \n    batch_size = 64,\n    sort_within_batch = True,\n    device = device)\n'

In [ ]:
def train(model, opt, criterion, dataloaders, max_epochs, patience=5):
  min_loss = np.inf

  cur_patience = 0
  patience = patience

  train_loader = dataloaders['train']
  valid_loader = dataloaders['val']

  for epoch in range(1, max_epochs + 1):
      train_loss = 0.0
      model.train()
      pbar = tqdm(enumerate(train_loader),
                  total=ceil(len(train_list)/batch_size),
                  leave=True)
      pbar.set_description(f"Epoch {epoch}")
      for it, batch in pbar:
        opt.zero_grad()
        labels, x = batch
    #   print('labels size:', labels.shape)
        labels, x = labels.to(device), x.to(device)
      #  print('labels type:', labels[0])
        output = model(x)[:, 0]
      #  print('output:', output[0])
        loss = criterion(output, labels.float())
        train_loss += loss.item()
        loss.backward()
        opt.step()

          #YOUR CODE GOES HERE

      train_loss /= ceil(len(train_list)/batch_size)

      val_loss = 0.0
      model.eval()
      pbar = tqdm(enumerate(valid_loader),
                  total=ceil(len(valid_list)/batch_size),
                  leave=True)
      pbar.set_description(f"Epoch {epoch}")
      with torch.no_grad():
        for it, batch in pbar:
          labels, x = batch
          labels, x = labels.to(device), x.to(device)
          output = model(x)[:, 0]
          val_loss += criterion(output, labels.float()).item()
            # YOUR CODE GOES HERE
        val_loss /= ceil(len(valid_list)/batch_size)
        if val_loss < min_loss:
            min_loss = val_loss
            best_model = model.state_dict()
        else:
            cur_patience += 1
            if cur_patience == patience:
                cur_patience = 0
                break

      print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
  model.load_state_dict(best_model)

In [ ]:
def test_model(model, test_loader):
  sigma = Sigmoid()
  y_true = []
  y_predicted = []
  model.eval()
  with torch.no_grad():
    for batch in test_loader:
      labels, x = batch
      labels, x = labels.to(device), x.to(device)
      for label in labels:
        y_true.append(label.item())
      y_pred = model(x)[:, 0]
      for pred in y_pred:
        logit = sigma(pred)
        y_predicted.append(logit.item() > 0.5)

  f1 = f1_score(y_true, y_predicted)
  return f1

## RNN

Для начала попробуем использовать рекурентные нейронные сети. На семинаре вы познакомились с GRU, вы можете также попробовать LSTM. Можно использовать для классификации как hidden_state, так и output последнего токена.

In [ ]:
#del model

In [ ]:
class RNNBaseline(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, pad_idx):

        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.n_layers = n_layers
        self.bidirectional = bidirectional
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        # YOUR CODE GOES HERE
        self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim,
                           num_layers=n_layers, batch_first=False,
                           bidirectional=self.bidirectional)

        self.fc = nn.Linear(hidden_dim * (1 + self.bidirectional),
                            self.output_dim)
       # self.fc = nn.Sequential(
       #     nn.Linear(hidden_dim * (1 + self.bidirectional), 128),
       #     nn.Linear(128, self.output_dim)
       # )  # YOUR CODE GOES HERE


    def forward(self, text, text_lengths):

        #text = [sent len, batch size]

        embedded = self.embedding(text)

        #embedded = [sent len, batch size, emb dim]

        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        h0 = torch.zeros(self.n_layers * (1 + self.bidirectional), embedded.size(1),
                         self.hidden_dim).to(device)
        z0 = torch.zeros(self.n_layers * (1 + self.bidirectional), embedded.size(1),
                         self.hidden_dim).to(device)
        # cell arg for LSTM, remove for GRU
        packed_output, (hidden, cell) = self.rnn(packed_embedded, (h0, z0))
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors

        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]

        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
      #  print('hidden shape before:', hidden.shape)
        hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)  # YOUR CODE GOES HERE
      #  print('hidden shape after:', hidden.shape)

        #hidden = [batch size, hid dim * num directions] or [batch_size, hid dim * num directions]
        return self.fc(hidden)

Поиграйтесь с гиперпараметрами

In [ ]:
vocab_size = len(vocabular)
emb_dim = 100
hidden_dim = 256
output_dim = 1
n_layers = 2
bidirectional = True
dropout = 0.2
PAD_IDX = vocabular['<pad>']
patience=3

In [ ]:
model = RNNBaseline(
    vocab_size=vocab_size,
    embedding_dim=emb_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    pad_idx=PAD_IDX
)

In [ ]:
model

RNNBaseline(
  (embedding): Embedding(27014, 100, padding_idx=2)
  (rnn): LSTM(100, 256, num_layers=2, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
)

In [ ]:
model = model.to(device)

In [ ]:
opt = torch.optim.Adam(model.parameters())
loss_func = nn.BCEWithLogitsLoss()

max_epochs = 20

Обучите сетку! Используйте любые вам удобные инструменты, Catalyst, PyTorch Lightning или свои велосипеды.

In [ ]:
# Попробовать переобучиться на одном батче
import numpy as np

min_loss = np.inf

cur_patience = 0
patience = patience

train_loader = dataloaders['train']
valid_loader = dataloaders['val']

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_loader),
                total=ceil(len(train_list)/batch_size),
                leave=True)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
      opt.zero_grad()
      labels, x = batch
    #   print('labels size:', labels.shape)
      labels, x = labels.to(device), x.to(device)
      lengths = [x.size(0)] * batch_size
      #  print('labels type:', labels[0])
      output = model(x, lengths)[:, 0]
      #  print('output:', output[0])
      loss = loss_func(output, labels.float())
      train_loss += loss.item()
      loss.backward()
      torch.nn.utils.clip_grad_norm(model.parameters(), max_grad=1)
      opt.step()

          #YOUR CODE GOES HERE

    train_loss /= ceil(len(train_list)/batch_size)

    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(valid_loader),
                total=ceil(len(valid_list)/batch_size),
                leave=True)
    pbar.set_description(f"Epoch {epoch}")
    with torch.no_grad():
      for it, batch in pbar:
        labels, x = batch
        labels, x = labels.to(device), x.to(device)
        lengths = [x.size(0)] * batch_size
        output = model(x, lengths)[:, 0]
        val_loss += loss_func(output, labels.float()).item()
            # YOUR CODE GOES HERE
      val_loss /= ceil(len(valid_list)/batch_size)
      if val_loss < min_loss:
          min_loss = val_loss
          best_model = model.state_dict()
      else:
          cur_patience += 1
          if cur_patience == patience:
              cur_patience = 0
              break

    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.6933427233457565, Validation Loss: 0.6932146936416625


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

...

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch: 3, Training Loss: 0.693178919672966, Validation Loss: 0.6934179160118104


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

<All keys matched successfully>

In [ ]:
sigma = Sigmoid()
y_true = []
y_predicted = []
model.eval()
with torch.no_grad():
  for batch in tqdm(test_loader):
    labels, x = batch
    labels, x = labels.to(device), x.to(device)
    lenghts = [x.size(0)] * batch_size
    for label in labels:
      y_true.append(label.item())
    y_pred = model(x, lenghts)[:, 0]
 #   print('y_pred:', y_pred)
    for pred in y_pred:
      logit = sigma(pred)
 #     print('logit:', logit)
      y_predicted.append(round(logit.item()))
print('y_true:', y_true)
print('y_predicted:', y_predicted)
f1 = f1_score(y_true, y_predicted)
print(f1)

  0%|          | 0/3125 [00:00<?, ?it/s]

y_true: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Посчитайте f1-score вашего классификатора на тестовом датасете.

**Ответ**: 0.6666666666666666

## CNN

![](https://www.researchgate.net/publication/333752473/figure/fig1/AS:769346934673412@1560438011375/Standard-CNN-on-text-classification.png)

Для классификации текстов также часто используют сверточные нейронные сети. Идея в том, что как правило сентимент содержат словосочетания из двух-трех слов, например "очень хороший фильм" или "невероятная скука". Проходясь сверткой по этим словам мы получим какой-то большой скор и выхватим его с помощью MaxPool. Далее идет обычная полносвязная сетка. Важный момент: свертки применяются не последовательно, а параллельно. Давайте попробуем!

Вы можете использовать Conv2d с `in_channels=1, kernel_size=(kernel_sizes[0], emb_dim))` или Conv1d c `in_channels=emb_dim, kernel_size=kernel_size[0]`. Но хорошенько подумайте над shape в обоих случаях.

In [ ]:
class CNN(nn.Module):
    def __init__(
        self,
        vocab_size,
        emb_dim,
        out_channels,
        kernel_sizes,
        dropout=0.5,
    ):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.conv_0 = nn.Sequential(
            nn.Conv1d(in_channels=emb_dim, out_channels=out_channels,
                                kernel_size=kernel_sizes[0], padding=1,
                                stride=2),  # YOUR CODE GOES HERE
            nn.BatchNorm1d(out_channels)
        )

        self.conv_1 = nn.Sequential(
            nn.Conv1d(in_channels=emb_dim, out_channels=out_channels,
                                kernel_size=kernel_sizes[1], padding=1,
                                stride=2),  # YOUR CODE GOES HERE
            nn.BatchNorm1d(out_channels)
        )


        self.conv_2 = nn.Sequential(
            nn.Conv1d(in_channels=emb_dim, out_channels=out_channels,
                                kernel_size=kernel_sizes[2], padding=1,
                                stride=2),  # YOUR CODE GOES HERE
            nn.BatchNorm1d(out_channels)
        )

        self.fc = nn.Linear(len(kernel_sizes) * out_channels, 1)

        self.dropout = nn.Dropout(dropout)


    def forward(self, text):
    #    print('shape before:', text.shape)
        embedded = self.embedding(text)
    #    print('shape after:', embedded.shape)

        embedded = embedded.permute(1, 2, 0)  # may be reshape here
    #    print('shape after permute:', embedded.shape)

        conved_0 = F.relu(self.conv_0(embedded))  # may be reshape here
        conved_1 = F.relu(self.conv_1(embedded))  # may be reshape here
        conved_2 = F.relu(self.conv_2(embedded))  # may be reshape here

        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)

        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))

        return self.fc(cat)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
#del model

In [ ]:
kernel_sizes = [3, 4, 5]
vocab_size = len(vocabular)
out_channels=64
dropout = 0.5
dim = 300

model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=out_channels,
            kernel_sizes=kernel_sizes, dropout=dropout)

In [ ]:
print(device)

cpu


In [ ]:
model.to(device)

CNN(
  (embedding): Embedding(27014, 300)
  (conv_0): Sequential(
    (0): Conv1d(300, 64, kernel_size=(3,), stride=(2,), padding=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_1): Sequential(
    (0): Conv1d(300, 64, kernel_size=(4,), stride=(2,), padding=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_2): Sequential(
    (0): Conv1d(300, 64, kernel_size=(5,), stride=(2,), padding=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc): Linear(in_features=192, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
opt = torch.optim.Adam(model.parameters())
loss_func = nn.BCEWithLogitsLoss()

In [ ]:
max_epochs = 30

###train

Обучите!

###Обучение с pytorch-lightning

In [ ]:
class ptlCNN(ptl.LightningModule):
  def __init__(self, cnn_model):
    super().__init__()
    self.cnn = cnn_model
    self.criterion = nn.BCEWithLogitsLoss()
    self.validation_step_outputs = []
    self.validation_true_labels = []
    self.test_step_outputs = []
    self.test_true_labels = []

  def training_step(self, batch, batch_idx):
    y, x = batch
    output = self.cnn(x)[:, 0]
    loss = self.criterion(output, y.float())
    self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True,
             logger = True)
    return loss

  def validation_step(self, batch, batch_idx):
    y, x = batch
    output = self.cnn(x)[:, 0]
    loss = self.criterion(output, y.float())
    self.log('valid_loss', loss, on_step=True, on_epoch=True, prog_bar=True,
             logger = True)
    for label in y:
  #    print('len val true labels:', len(self.validation_true_labels))
      self.validation_true_labels.append(label.item())
    for pred in output:
  #    print('len val step outputs:', len(self.validation_step_outputs))
      logit = torch.sigmoid(pred)
      self.validation_step_outputs.append(logit.item() > 0.5)
    return loss

  def on_validation_epoch_end(self):
    all_preds = self.validation_step_outputs
    all_true = self.validation_true_labels
 #   print('len all_preds:', len(all_preds))
 #   print('len all_true:', len(all_true))
    print(f'Validation accuracy = {sum([all_preds[i] == all_true[i] for i in range(len(all_true))]) / len(all_true)}')
    self.validation_step_outputs.clear()
    self.validation_true_labels.clear()

  def test_step(self, batch, batch_idx):
    y, x = batch
    output = self.cnn(x)[:, 0]
    loss = self.criterion(output, y.float())
    self.log('test_loss', loss, on_step=True, on_epoch=True, prog_bar=True,
             logger = True)
    for label in y:
      self.test_true_labels.append(label.item())
    for pred in output:
      logit = torch.sigmoid(pred)
      self.test_step_outputs.append(logit.item() > 0.5)
    return loss

  def on_test_epoch_end(self):
    all_preds = self.test_step_outputs
    all_true = self.test_true_labels
    print(f'testing f1 score = {f1_score(all_true, all_preds)}')
    self.test_step_outputs.clear()
    self.test_true_labels.clear()

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=1e-3)








In [ ]:
trainer = ptl.Trainer(max_epochs=1)
ptl_model = ptlCNN(model)
trainer.fit(ptl_model, dataloaders['train'], dataloaders['val'])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type              | Params
------------------------------------------------
0 | cnn       | CNN               | 8.3 M 
1 | criterion | BCEWithLogitsLoss | 0     
------------------------------------------------
8.3 M     Trainable params
0         Non-trainable params
8.3 M     Total params
33.341    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Validation accuracy = 0.9375


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation accuracy = 0.8884


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
trainer.test(dataloaders=test_loader, ckpt_path='best')

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_6/checkpoints/epoch=0-step=2500.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_6/checkpoints/epoch=0-step=2500.ckpt


Testing: 0it [00:00, ?it/s]

testing f1 score = 0.8889410843609317


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_loss_epoch      │    0.4012085199356079     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_epoch': 0.4012085199356079}]

###Обычный трейн

In [ ]:
def train(model, opt, criterion, dataloaders, max_epochs, patience=5):
  min_loss = np.inf

  cur_patience = 0
  patience = patience

  train_loader = dataloaders['train']
  valid_loader = dataloaders['val']

  for epoch in range(1, max_epochs + 1):
      train_loss = 0.0
      model.train()
      pbar = tqdm(enumerate(train_loader),
                  total=ceil(len(train_list)/batch_size),
                  leave=True)
      pbar.set_description(f"Epoch {epoch}")
      for it, batch in pbar:
        opt.zero_grad()
        labels, x = batch
    #   print('labels size:', labels.shape)
        labels, x = labels.to(device), x.to(device)
      #  print('labels type:', labels[0])
        output = model(x)[:, 0]
      #  print('output:', output[0])
        loss = criterion(output, labels.float())
        train_loss += loss.item()
        loss.backward()
        opt.step()

          #YOUR CODE GOES HERE

      train_loss /= ceil(len(train_list)/batch_size)

      val_loss = 0.0
      model.eval()
      pbar = tqdm(enumerate(valid_loader),
                  total=ceil(len(valid_list)/batch_size),
                  leave=True)
      pbar.set_description(f"Epoch {epoch}")
      with torch.no_grad():
        for it, batch in pbar:
          labels, x = batch
          labels, x = labels.to(device), x.to(device)
          output = model(x)[:, 0]
          val_loss += criterion(output, labels.float()).item()
            # YOUR CODE GOES HERE
        val_loss /= ceil(len(valid_list)/batch_size)
        if val_loss < min_loss:
            min_loss = val_loss
            best_model = model.state_dict()
        else:
            cur_patience += 1
            if cur_patience == patience:
                cur_patience = 0
                break

      print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
  model.load_state_dict(best_model)

In [ ]:
def test_model(model, test_loader):
  sigma = Sigmoid()
  y_true = []
  y_predicted = []
  model.eval()
  with torch.no_grad():
    for batch in test_loader:
      labels, x = batch
      labels, x = labels.to(device), x.to(device)
      for label in labels:
        y_true.append(label.item())
      y_pred = model(x)[:, 0]
      for pred in y_pred:
        logit = sigma(pred)
        y_predicted.append(logit.item() > 0.5)

  f1 = f1_score(y_true, y_predicted)
  return f1


###Подбор гиперпараметров с помощью Wandb

In [ ]:
!pip install -q wandb

In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: nalex22-uv-j02. Use `wandb login --relogin` to force relogin


True

In [ ]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'f1-score',
        'goal': 'maximize'
    }
}
parameters_dict = {
    'optimizer': {
        'values': ['Adam', 'AdamW']
    },
    'criterion': {
        'value': 'BCEWithLogitsLoss'
    },
    'epochs': {
        'value': 30
    },
    'patience': {
        'value': 7
    },
    'learning_rate': {
        'distribution': 'uniform',
        'min': 0.0001,
        'max': 0.01
    },
    'batch_size': {
        'distribution': 'q_log_uniform_values',
        'q': 2,
        'min': 4,
        'max': 30
    },
    'out_channels': {
        'distribution': 'int_uniform',
        'min': 40,
        'max': 80
    },
    'dropout': {
        'distribution': 'uniform',
        'min': 0.1,
        'max': 0.6
    },
    'kernel_sizes': {
        'value': [3, 4, 5]
    },
    'embedding_dim': {
        'value': 300
    },
    'vocab_size': {
        'value': len(vocabular)
    }
}
sweep_config['parameters'] = parameters_dict

In [ ]:
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'f1-score'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 30,
                               'min': 4,
                               'q': 2},
                'criterion': {'value': 'BCEWithLogitsLoss'},
                'dropout': {'distribution': 'uniform', 'max': 0.6, 'min': 0.1},
                'embedding_dim': {'value': 300},
                'epochs': {'value': 30},
                'kernel_sizes': {'value': [3, 4, 5]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.01,
                                  'min': 0.0001},
                'optimizer': {'values': ['Adam', 'AdamW']},
                'out_channels': {'distribution': 'int_uniform',
                                 'max': 80,
                                 'min': 40},
                'patience': {'value': 7},
                'vocab_size': {'va

In [ ]:
sweep_id = wandb.sweep(sweep_config, project='text-classification-CNN_baseline-sweep-hp')

Create sweep with ID: hp2q8g1t
Sweep URL: https://wandb.ai/nalex22-uv-j02/text-classification-CNN_baseline-sweep-hp/sweeps/hp2q8g1t


In [ ]:
weights_dir = 'model_weights'
if not os.path.exists(weights_dir):
  os.mkdir(weights_dir)

In [ ]:
def train(config=None):
  with wandb.init(config=config):
    cur_patience = 0
    max_f1 = -np.inf
    #average_val_loss = 0
    config = wandb.config
    patience = config.patience
    loader = build_dataset(config.batch_size)
    network = build_network(config.vocab_size, config.embedding_dim,
                            config.out_channels, config.kernel_sizes,
                            config.dropout)
    criterion = build_criterion(config.criterion)
    optimizer = build_optimizer(network, config.optimizer, config.learning_rate,
                                )
   # scheduler = build_lr_scheduler(optimizer, config.lr_scheduler,
   #                                config.gamma, config.step_size)
    for epoch in trange(config.epochs):
      val_f1 = train_epoch(network, loader, optimizer,
                                      criterion)
      if val_f1 > max_f1:
        max_f1 = val_f1
        best_model_weights = network.state_dict()
      else:
        cur_patience += 1
        if cur_patience == patience:
          cur_patience = 0
          break

     # wandb.log({'accuracy': avg_acc_per_epoch, 'loss': avg_loss})
    wandb.log({'f1-score': max_f1})
    torch.save(best_model_weights, f'{weights_dir}/f1={max_f1}.pt')
    del network

In [ ]:
def build_dataset(batch_size):
  #dataset = load_dataset("FastJobs/Visual_Emotional_Analysis")
  #train_val_set = dataset['train'].train_test_split(test_size=0.15, shuffle=True, stratify_by_column='label')
  #train_val_set.set_format(type='torch', dtype=torch.float)

  dataloaders = {
               k: DataLoader(
                   data_list,
                   batch_size=batch_size,
                   collate_fn=collate_batch) for k, data_list in zip(['train', 'val'],
                                                                     [train_list, valid_list])
}
  return dataloaders

def build_network(vocab_size, emb_dim, out_channels, kernel_sizes, dropout):
  model = CNN(vocab_size, emb_dim, out_channels, kernel_sizes, dropout)
  return model.to(device)


def build_optimizer(model, name, lr):
  if name == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  elif name == 'AdamW':
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
  else:
    print('something wrong with optimizer name')
    return None
  return optimizer

def build_criterion(name):
  if name == 'BCEWithLogitsLoss':
    criterion = nn.BCEWithLogitsLoss()
  else:
    print('something wrong with criterion name')
    return None
  return criterion

def train_epoch(model, loaders, optimizer, criterion):
  train_loader = loaders['train']
  val_loader = loaders['val']
  avg_train_loss = 0
  model.train()
  for batch in tqdm(train_loader):
    labels, x = batch
    labels, x = labels.to(device), x.to(device)
    optimizer.zero_grad()
    prediction = model(x)[:, 0]
    loss = criterion(prediction, labels.float())
    loss.backward()
    optimizer.step()
    avg_train_loss += loss.item()
    del x, labels
   # wandb.log({'batch loss': loss.item()})
  avg_train_loss /= len(train_loader)

  model.eval()
  with torch.no_grad():
    f1, num_samples, avg_val_loss = 0, 0, 0
    y_true, y_predicted = [], []
    for batch in tqdm(val_loader):
      labels, x = batch
      labels, x = labels.to(device), x.to(device)
      for label in labels:
        y_true.append(label.item())
      prediction = model(x)[:, 0]
      for pred in prediction:
        logit = torch.sigmoid(pred)
        y_predicted.append(logit.item() > 0.5)
      avg_val_loss += criterion(prediction, labels.float()).item()
      del x, labels
    f1 = f1_score(y_true, y_predicted)
    avg_val_loss /= len(val_loader)

    wandb.log({'avg_train_loss': avg_train_loss, 'avg_val_loss': avg_val_loss,
               'val_f1_score': f1})
    return f1

In [ ]:
wandb.agent(sweep_id, train, count=10)

wandb: Agent Starting Run: 98q19ww7 with config:
wandb: 	batch_size: 30
wandb: 	criterion: BCEWithLogitsLoss
wandb: 	dropout: 0.2643279938709725
wandb: 	embedding_dim: 300
wandb: 	epochs: 30
wandb: 	kernel_sizes: [3, 4, 5]
wandb: 	learning_rate: 0.0010127385077656928
wandb: 	optimizer: AdamW
wandb: 	out_channels: 72
wandb: 	patience: 7
wandb: 	vocab_size: 27014


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/667 [00:00<?, ?it/s]

  0%|          | 0/167 [00:00<?, ?it/s]

  0%|          | 0/667 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

avg_train_loss,█▅▃▂▂▁▁▁▁
avg_val_loss,▃▁▃▅▆▆▆██
f1-score,▁
val_f1_score,▁█▅▄▂▃▁▃▁
avg_train_loss,0.13774
avg_val_loss,0.49146
f1-score,0.86281
val_f1_score,0.84698


wandb: Agent Starting Run: bku61cgo with config:
wandb: 	batch_size: 16
wandb: 	criterion: BCEWithLogitsLoss
wandb: 	dropout: 0.1316843009754797
wandb: 	embedding_dim: 300
wandb: 	epochs: 30
wandb: 	kernel_sizes: [3, 4, 5]
wandb: 	learning_rate: 0.009924718809452582
wandb: 	optimizer: Adam
wandb: 	out_channels: 78
wandb: 	patience: 7
wandb: 	vocab_size: 27014


In [ ]:
model = CNN(vocab_size=27014, emb_dim=300, out_channels=72,
            kernel_sizes=[3, 4, 5], dropout=0.2643279938709725).to(device)

In [ ]:
model.load_state_dict(torch.load('f1=0,897196261682243.pt', map_location=device))

<All keys matched successfully>

In [ ]:
model.embedding.weight[0]

tensor([-0.2759, -0.4310,  0.1196, -0.6288,  0.1498,  0.3668,  0.1384, -0.5576,
        -0.2588,  0.6224, -0.6340, -0.2852, -0.5055, -1.4812,  0.2627, -0.9688,
         0.8461,  0.0585, -0.9844, -1.2281, -0.0437,  0.2216, -0.5333, -0.5194,
        -0.1893, -1.0405,  0.3353,  0.2381, -0.9156,  0.7623, -0.2215,  1.0019,
        -1.4375,  0.7144, -1.1718, -0.9137, -0.5071, -0.1711, -1.8144,  0.3667,
         0.9235, -0.0228, -0.2956, -1.0490,  1.3187, -0.7134,  0.5679,  0.1469,
         1.5462, -0.1480, -1.1459, -1.6866, -1.3882,  0.4729, -1.0651,  0.4831,
         1.1146,  1.6416,  0.3307,  0.8677, -0.0838, -1.4424, -1.6608,  0.6842,
         0.6107,  0.6269,  0.2958, -0.1660,  0.5661, -1.1295,  0.1840,  0.5711,
        -0.1609, -0.3666, -1.7489,  0.5569,  0.6646, -0.1153, -0.8829,  0.8112,
        -0.1547, -1.1136, -0.7449, -0.6199,  0.2961,  0.3597, -0.2738,  0.1646,
         0.6463, -1.1500, -0.2072, -0.4689,  0.2533,  2.2183, -1.5417,  0.4380,
         0.7343,  1.5613, -0.0880, -0.30

In [ ]:
print(test_model(model, test_loader))

0.8813055153893824


###что-то левое

In [ ]:
'''
min_loss = np.inf

cur_patience = 0
patience = 5

train_loader = dataloaders['train']
valid_loader = dataloaders['val']

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_loader),
                total=ceil(len(train_list)/batch_size),
                leave=True)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
      opt.zero_grad()
      labels, x = batch
   #   print('labels size:', labels.shape)
      labels, x = labels.to(device), x.to(device)
    #  print('labels type:', labels[0])
      output = model(x)[:, 0]
    #  print('output:', output[0])
      loss = loss_func(output, labels.float())
      train_loss += loss.item()
      loss.backward()
      opt.step()

        #YOUR CODE GOES HERE

    train_loss /= ceil(len(train_list)/batch_size)

    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(valid_loader),
                total=ceil(len(valid_list)/batch_size),
                leave=True)
    pbar.set_description(f"Epoch {epoch}")
    with torch.no_grad():
      for it, batch in pbar:
        labels, x = batch
        labels, x = labels.to(device), x.to(device)
        output = model(x)[:, 0]
        val_loss += loss_func(output, labels.float()).item()
          # YOUR CODE GOES HERE
      val_loss /= ceil(len(valid_list)/batch_size)
      if val_loss < min_loss:
          min_loss = val_loss
          best_model = model.state_dict()
      else:
          cur_patience += 1
          if cur_patience == patience:
              cur_patience = 0
              break

    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)
'''

In [ ]:
'''
sigma = Sigmoid()
y_true = []
y_predicted = []
model.eval()
with torch.no_grad():
  for batch in test_loader:
    labels, x = batch
    labels, x = labels.to(device), x.to(device)
    for label in labels:
      y_true.append(label.item())
    y_pred = model(x)[:, 0]
    for pred in y_pred:
      logit = sigma(pred)
      y_predicted.append(logit.item() > 0.5)

f1 = f1_score(y_true, y_predicted)
print('f1_score:', f1)
'''

Выходные данные были обрезаны до нескольких последних строк (5000).
shape after permute: torch.Size([8, 300, 1174])
shape before: torch.Size([835, 8])
shape after: torch.Size([835, 8, 300])
shape after permute: torch.Size([8, 300, 835])
shape before: torch.Size([909, 8])
shape after: torch.Size([909, 8, 300])
shape after permute: torch.Size([8, 300, 909])
shape before: torch.Size([911, 8])
shape after: torch.Size([911, 8, 300])
shape after permute: torch.Size([8, 300, 911])
shape before: torch.Size([249, 8])
shape after: torch.Size([249, 8, 300])
shape after permute: torch.Size([8, 300, 249])
shape before: torch.Size([386, 8])
shape after: torch.Size([386, 8, 300])
shape after permute: torch.Size([8, 300, 386])
shape before: torch.Size([730, 8])
shape after: torch.Size([730, 8, 300])
shape after permute: torch.Size([8, 300, 730])
shape before: torch.Size([478, 8])
shape after: torch.Size([478, 8, 300])
shape after permute: torch.Size([8, 300, 478])
shape before: torch.Size([317, 8])
sh

Посчитайте f1-score вашего классификатора.

**Ответ**: 0.8891005795030563

## Интерпретируемость

Посмотрим, куда смотрит наша модель. Достаточно запустить код ниже.

In [ ]:
vocabular['<pad>']

2

In [ ]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

PAD_IND = vocabular['<pad>']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)

In [ ]:
def forward_with_softmax(inp):
    logits = model(inp)
    return torch.softmax(logits, 0)[0][1]

def forward_with_sigmoid(input):
    return torch.sigmoid(model(input)[:, 0])


# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 7, label = 0):
    model.eval()
    text = [tok for tok in tokenizer(sentence)]
    if len(text) < min_len:
        text += ['<pad>'] * (min_len - len(text))
    indexed = [new_stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(1)

  #  print('input indices shape:', input_indices.shape)

    # input_indices dim: [sequence_length]
    seq_length = input_indices.shape[0]


    # predict
    pred = forward_with_sigmoid(input_indices).item()
    pred_ind = round(pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(1)
  #  print('reference_indices shape:', reference_indices.shape)
  #  print('reference_indices:', reference_indices)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=5000, return_convergence_delta=True)
  #  print('attributions_ig:', attributions_ig)

    print('pred: ', labels_to_text[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)

def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
  #  print('attributions shape before :', attributions.shape)
  #  print('attributions itself before:', attributions)
    attributions = attributions.sum(dim=2).squeeze(0)
  #  print('attributions shape after:', attributions.shape)
  #  print('attributions itself after:', attributions)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            labels_to_text[pred_ind],
                            labels_to_text[label],
                            pred_ind,
                            attributions.sum(),
                            text,
                            delta))

In [ ]:
interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.65 ) , delta:  tensor([0.2178, 0.2178, 0.2178, 0.2178, 0.2178, 0.2178, 0.2178],
       dtype=torch.float64)
attributions shape before : torch.Size([7, 1, 300])
attributions itself before: tensor([[[-2.2361e-10, -2.3008e-09, -1.8583e-09,  ...,  2.1417e-09,
          -2.9180e-10,  4.9991e-09]],

        [[-7.3568e-10, -4.1181e-09,  1.9023e-09,  ..., -1.9963e-09,
          -4.5052e-09, -2.4917e-09]],

        [[ 5.1481e-09, -3.4932e-09,  3.7363e-09,  ...,  2.6651e-09,
          -1.1897e-10, -4.3632e-09]],

        ...,

        [[-1.0104e-09, -5.8766e-09, -3.6906e-10,  ...,  1.3050e-08,
          -5.4341e-10, -2.0533e-09]],

        [[-1.2245e-10,  1.9436e-09,  8.6394e-10,  ..., -6.6040e-09,
          -2.7373e-10,  4.0378e-09]],

        [[ 0.0000e+00,  0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
          -0.0000e+00,  0.0000e+00]]], dtype=torch.float64)
attributions shape after: torch.Size([7, 1])
attributions itself after: tensor([[ 9.2005e-08],
        [ 4.9238e-08],
  

Попробуйте добавить свои примеры!

In [ ]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (1.00),pos,1.72,it was a fantastic performance ! #pad
pos,pos (0.79),pos,1.35,best film ever #pad #pad #pad #pad
pos,pos (0.96),pos,1.28,such a great show ! #pad #pad
neg,neg (0.00),pos,-1.29,it was a horrible movie #pad #pad
neg,neg (0.46),pos,-0.20,i ' ve never watched something as bad
neg,neg (0.12),pos,-0.82,it is a disgusting movie ! #pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (1.00),pos,1.72,it was a fantastic performance ! #pad
pos,pos (0.79),pos,1.35,best film ever #pad #pad #pad #pad
pos,pos (0.96),pos,1.28,such a great show ! #pad #pad
neg,neg (0.00),pos,-1.29,it was a horrible movie #pad #pad
neg,neg (0.46),pos,-0.20,i ' ve never watched something as bad
neg,neg (0.12),pos,-0.82,it is a disgusting movie ! #pad


## Эмбеддинги слов

Вы ведь не забыли, как мы можем применить знания о word2vec и GloVe. Давайте попробуем!

In [ ]:
word_embeddings = GloVe(max_vectors=200000)

100%|█████████▉| 199999/200000 [00:26<00:00, 7472.89it/s]


In [ ]:
word_embeddings['addicted']

tensor([ 1.9703e-01,  1.5457e-01, -3.5752e-01,  4.7194e-02, -4.4737e-01,
        -4.0381e-01,  4.3476e-02, -3.2467e-01,  2.3264e-01,  1.4357e+00,
        -7.0707e-01,  2.8149e-01,  1.0325e-01, -3.0205e-01,  3.2867e-01,
        -4.6170e-01, -4.0728e-01,  4.8592e-02,  1.5475e-01,  8.4592e-01,
        -1.4383e-01, -2.1955e-01, -5.5943e-01,  2.6859e-01, -3.9332e-01,
        -1.7264e-02,  5.4889e-01, -2.0476e-01,  4.9640e-01, -3.7663e-01,
        -3.1947e-02,  8.2853e-02, -2.8719e-01,  8.2484e-02, -3.1018e-01,
         3.9550e-02, -5.5911e-01,  3.3474e-01, -5.3821e-01,  3.7378e-01,
         3.0666e-02, -2.7170e-01,  7.8092e-01,  2.3497e-01, -4.2438e-01,
         1.9685e-01,  5.5733e-02,  2.9911e-01,  3.6586e-01,  2.1472e-01,
        -3.6891e-01, -3.7051e-02,  1.0712e-01, -1.2861e-01, -1.9569e-01,
        -5.5735e-02, -3.7074e-01, -4.7546e-01, -2.4042e-01, -4.2223e-01,
        -9.6036e-02, -5.4337e-01, -3.7576e-01, -2.6236e-01, -1.9652e-02,
        -2.2570e-01,  3.0483e-01, -6.8769e-01,  3.7

In [ ]:
len(word_embeddings)

200000

In [ ]:
#TEXT.build_vocab(trn, vectors=)# YOUR CODE GOES HERE
# подсказка: один из импортов пока не использовался, быть может он нужен в строке выше :)
#LABEL.build_vocab(trn)

#word_embeddings = TEXT.vocab.vectors

kernel_sizes = [3, 4, 5]
vocab_size = len(word_embeddings) #len(TEXT.vocab)
dropout = 0.5
dim = 300

In [ ]:
#train, tst = datasets.IMDB.splits(TEXT, LABEL)
#trn, vld = train.split(random_state=random.seed(SEED))
'''
device = "cuda" if torch.cuda.is_available() else "cpu"
        return self.fc(hidden)
train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)
'''

In [ ]:
model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=64,
            kernel_sizes=kernel_sizes, dropout=dropout)

In [ ]:
len(word_embeddings.itos)

200000

In [ ]:
with torch.no_grad():
  for idx, word in enumerate(word_embeddings.itos):
    model.embedding.weight[idx] = word_embeddings[word]

In [ ]:
#model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=64,
#            kernel_sizes=kernel_sizes, dropout=dropout)

#word_embeddings = TEXT.vocab.vectors

#prev_shape = model.embedding.weight.shape

#model.embedding.weight = # инициализируйте эмбэдинги

#assert prev_shape == model.embedding.weight.shape
model.to(device)

max_epochs = 30

opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_func = nn.BCEWithLogitsLoss()

In [ ]:
#del model

Вы знаете, что делать.

In [ ]:
def freeze_embeddings(model, req_grad=False):
    embeddings = model.embedding
    for c_p in embeddings.parameters():
        c_p.requires_grad = req_grad

In [ ]:
def training_freeze(model, opt, criterion, max_epochs, dataloaders,
                    patience=5, num_freeze_iter=2000):
  freeze_embeddings(model)
  min_loss = np.inf
  cur_patience = 0
  train_loader = dataloaders['train']
  valid_loader = dataloaders['val']
  num_iter = 0

  for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_loader),
                total=ceil(len(train_list)/batch_size),
                leave=True)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
      if num_iter > num_freeze_iter and epoch < 1:
        freeze_embeddings(model, True)
      opt.zero_grad()
      labels, x = batch
   #   print('labels size:', labels.shape)
      labels, x = labels.to(device), x.to(device)
    #  print('labels type:', labels[0])
      output = model(x)[:, 0]
    #  print('output:', output[0])
      loss = criterion(output, labels.float())
      train_loss += loss.item()
      loss.backward()
      opt.step()
      num_iter += 1

        #YOUR CODE GOES HERE

    train_loss /= ceil(len(train_list)/batch_size)

    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(valid_loader),
                total=ceil(len(valid_list)/batch_size),
                leave=True)
    pbar.set_description(f"Epoch {epoch}")
    with torch.no_grad():
      for it, batch in pbar:
        labels, x = batch
        labels, x = labels.to(device), x.to(device)
        output = model(x)[:, 0]
        val_loss += criterion(output, labels.float()).item()
          # YOUR CODE GOES HERE
      val_loss /= ceil(len(valid_list)/batch_size)
      if val_loss < min_loss:
          min_loss = val_loss
          best_model = model.state_dict()
      else:
          cur_patience += 1
          if cur_patience == patience:
              cur_patience = 0
              break

    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch,
                                                                     train_loss,
                                                                     val_loss))
  model.load_state_dict(best_model)
  return model



In [ ]:
model = training_freeze(model, opt, loss_func, max_epochs, dataloaders,
                num_freeze_iter=1)

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.6915912386655807, Validation Loss: 0.598146035528183


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

...

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch: 17, Training Loss: 0.19730061003346927, Validation Loss: 0.37096694210767744


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

In [ ]:

from math import ceil

max_epochs = 30
min_loss = np.inf

cur_patience = 0
patience = 5

train_loader = dataloaders['train']
valid_loader = dataloaders['val']

#freeze_embeddings(model)
num_iter = 0
num_freeze_iter = 500
for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_loader),
                total=ceil(len(train_list)/batch_size),
                leave=True)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
      #if num_iter > num_freeze_iter and epoch < 1:
      #  freeze_embeddings(model, True)
      opt.zero_grad()
      labels, x = batch
   #   print('labels size:', labels.shape)
      labels, x = labels.to(device), x.to(device)
    #  print('labels type:', labels[0])
      output = model(x)[:, 0]
    #  print('output:', output[0])
      loss = loss_func(output, labels.float())
      train_loss += loss.item()
      loss.backward()
      opt.step()

        #YOUR CODE GOES HERE

    train_loss /= ceil(len(train_list)/batch_size)

    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(valid_loader),
                total=ceil(len(valid_list)/batch_size),
                leave=True)
    pbar.set_description(f"Epoch {epoch}")
    with torch.no_grad():
      for it, batch in pbar:
        labels, x = batch
        labels, x = labels.to(device), x.to(device)
        output = model(x)[:, 0]
        val_loss += loss_func(output, labels.float()).item()
          # YOUR CODE GOES HERE
      val_loss /= ceil(len(valid_list)/batch_size)
      if val_loss < min_loss:
          min_loss = val_loss
          best_model = model.state_dict()
      else:
          cur_patience += 1
          if cur_patience == patience:
              cur_patience = 0
              break

    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)


  0%|          | 0/2500 [00:00<?, ?it/s]

In [ ]:
'''
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
        #YOUR CODE GOES HERE

    train_loss /= len(train_iter)
    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(valid_iter), total=len(valid_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
        # YOUR CODE GOES HERE
    val_loss /= len(valid_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break

    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)
'''

In [ ]:
f1 = test_model(model, test_loader)
print('f1_score:', f1)

f1_score: 0.8456904358782533


Посчитайте f1-score вашего классификатора.

**Ответ**: 0.8843603823469596

Проверим насколько все хорошо!

In [ ]:
PAD_IND = vocabular['<pad>']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)
vis_data_records_ig = []


interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 1.00 ) , delta:  tensor([7.0405, 7.0405, 7.0405, 7.0405, 7.0405, 7.0405, 7.0405],
       device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.66 ) , delta:  tensor([1.9748, 1.9748, 1.9748, 1.9748, 1.9748, 1.9748, 1.9748],
       device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.96 ) , delta:  tensor([4.5053, 4.5053, 4.5053, 4.5053, 4.5053, 4.5053, 4.5053],
       device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([6.9268, 6.9268, 6.9268, 6.9268, 6.9268, 6.9268, 6.9268],
       device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.11 ) , delta:  tensor([0.7635, 0.7635, 0.7635, 0.7635, 0.7635, 0.7635, 0.7635, 0.7635],
       device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.21 ) , delta:  tensor([0.0115, 0.0115, 0.0115, 0.0115, 0.0115, 0.0115, 0.0115],
       device='cuda:0', dtype=torch.float64)


In [ ]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (1.00),1,1.58,it was a fantastic performance ! #pad
pos,pos (0.66),1,1.49,best film ever #pad #pad #pad #pad
pos,pos (0.96),1,1.28,such a great show ! #pad #pad
neg,neg (0.00),0,-0.87,it was a horrible movie #pad #pad
neg,neg (0.11),0,-0.27,i ' ve never watched something as bad
neg,neg (0.21),0,0.49,it is a disgusting movie ! #pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (1.00),1,1.58,it was a fantastic performance ! #pad
pos,pos (0.66),1,1.49,best film ever #pad #pad #pad #pad
pos,pos (0.96),1,1.28,such a great show ! #pad #pad
neg,neg (0.00),0,-0.87,it was a horrible movie #pad #pad
neg,neg (0.11),0,-0.27,i ' ve never watched something as bad
neg,neg (0.21),0,0.49,it is a disgusting movie ! #pad
